In [2]:
import numpy as np
import cv2

In [2]:
## 한글사용
from PIL import ImageFont, ImageDraw, Image

img = cv2.imread('image/iron.jpg',1)
img = Image.fromarray(img) #img배열을 PIL이 처리가능하게 변환

draw = ImageDraw.Draw(img)
font=ImageFont.truetype('C:\Windows\Fonts/batang.ttc',20)  
draw.text((100,300),"안녕하세요 \n반갑습니다",font = font,fill = (0,55,0)) #text를 출력 
img = np.array(img) #다시 OpenCV가 처리가능하게 np 배열로 변환

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
## 모폴로지 연산 

morph_dot = cv2.imread('image/morph_dot.png',0)
morph_dot = cv2.resize(morph_dot, dsize = (0,0), fx = 2, fy = 2, interpolation = cv2.INTER_LINEAR)

morph_hole = cv2.imread('image/morph_hole.png',0)
morph_hole = cv2.resize(morph_hole, dsize = (0,0), fx = 2, fy = 2, interpolation = cv2.INTER_LINEAR)

morphology = cv2.imread('image/morphological.png',0)
morphology = cv2.resize(morphology, dsize = (0,0), fx = 2, fy = 2, interpolation = cv2.INTER_LINEAR)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))

morph_open = cv2.morphologyEx(morph_dot, cv2.MORPH_OPEN, kernel, iterations = 1)
morph_close = cv2.morphologyEx(morph_hole, cv2.MORPH_CLOSE, kernel, iterations = 1)
## morph_gradient = cv2.morphologyEx(morphology, cv2.MORPH_GRADIENT, kernel, iterations = 1)
morph_tophat = cv2.morphologyEx(morph_dot, cv2.MORPH_TOPHAT, kernel, iterations = 1)
morph_blackhat = cv2.morphologyEx(morph_hole, cv2.MORPH_BLACKHAT, kernel, iterations = 1)
## morph_hitmiss = cv2.morphologyEx(morph_hole, cv2.MORPH_HITMISS, kernel, iterations = 1)

_open = np.concatenate((morph_dot, morph_open), axis = 0)
_close = np.concatenate((morph_hole, morph_close), axis = 0)
## _gradient = np.concatenate((morphology,morph_gradient), axis = 0)
_tophat = np.concatenate((morph_dot, morph_tophat), axis = 0)
_blackhat = np.concatenate((morph_hole, morph_blackhat), axis = 0)
## _hitmiss = np.concatenate((morph_hole, morph_hitmiss), axis = 0)

morph = np.concatenate((_open, _close, _tophat, _blackhat), axis = 1)
                           
cv2.imshow('morph_ex', morph)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
## 직선 검출
src = cv2.imread('image/road.jpg', 1)
src = cv2.resize(src, dsize = (0,0), fx = 0.8, fy = 0.8,interpolation = cv2.INTER_AREA)
dst = src.copy()
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(gray, 5000, 1500, apertureSize = 5, L2gradient = True)
lines = cv2.HoughLines(canny, 0.8, np.pi / 180, 150, min_theta = 0, max_theta = np.pi)

for i in lines:
    rho, theta = i[0][0], i[0][1]
    a, b = np.cos(theta), np.sin(theta)
    x0, y0 = a*rho, b*rho
    
    scale = src.shape[0] + src.shape[1]
    
    x1 = int(x0 + scale * -b)
    y1 = int(y0 + scale * a)
    x2 = int(x0 - scale * -b)
    y2 = int(y0 - scale * a)
    
    cv2.line(dst,(x1,y1),(x2,y2),(0,0,255),2)
    cv2.circle(dst,(x0.astype(np.uint16),y0.astype(np.uint16)),3, (255,0,0), 5,cv2.FILLED)
    ## circle 함수는 중앙점을 대입할 때 자료형이 uint16 이여야 함.
    
    
cv2.imshow('canny',canny)   
cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
##점진성 확률적 허프 변환

src = cv2.imread('image/road.jpg', 1)
src = cv2.resize(src, dsize = (0,0), fx = 0.8, fy = 0.8,interpolation = cv2.INTER_AREA)
dst = src.copy()
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(gray, 5000, 1500, apertureSize = 5, L2gradient = True)
lines = cv2.HoughLinesP(canny, 0.8, np.pi / 180, 90, minLineLength = 10, maxLineGap = 100)

for i in lines:
    cv2.line(dst, (i[0][0], i[0][1]), (i[0][2], i[0][3]), (0, 0, 255), 2)

cv2.imshow("dst", dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
print(lines)

[[[1131  671 1535  904]]

 [[ 109  748  776  437]]

 [[1026  616 1535  921]]

 [[ 668  435 1134  673]]

 [[ 144  723  780  439]]

 [[1276  755 1322  780]]

 [[1020  613 1533  921]]

 [[ 657  890  698  635]]

 [[1214  728 1532  919]]

 [[ 684  891  723  518]]

 [[  71  767  177  717]]

 [[ 839  523 1116  671]]

 [[ 336  641  590  527]]

 [[ 669  435 1113  661]]

 [[   2  792  149  721]]

 [[ 216  690  777  440]]

 [[ 690  842  722  537]]

 [[ 646  936  723  498]]

 [[1268  749 1495  880]]]


In [3]:
## 원검출 resize 했을 때 검출되는 원이 다르므로 피라미터 수정해야댐

src = cv2.imread('image/colorball.jpg', 1)
##src = cv2.resize(src, dsize = (0,0), fx = 0.5, fy= 0.5, interpolation = cv2.INTER_AREA)
dst = src.copy()
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)


circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 50, param1 = 230, param2 = 10, minRadius = 40, maxRadius = 80)

for i in circles[0]:
    cv2.circle(dst,(i[0].astype(np.uint16),i[1].astype(np.uint16)),i[2].astype(np.uint16), (255,255,255), 5)
    
    
cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
## 이미지 연산 (더하기 빼기 곱하기 나누기)

src = cv2.imread('image/pencil.jpg', 1)
src = cv2.resize(src, dsize = (0,0), fx= 0.2, fy = 0.2, interpolation = cv2.INTER_AREA)
number1 = np.ones_like(src)*127
number2 = np.ones_like(src)*2

add = cv2.add(src, number1)
sub = cv2.subtract(src, number1)
mul = cv2.multiply(src, number2)
div = cv2.divide(src, number2)

src = np.concatenate((src,src,src,src), axis = 1)
number = np.concatenate((number1,number1,number2,number2), axis = 1)
dst = np.concatenate((add, sub, mul, div), axis = 1)

dst = np.concatenate((src,number,dst), axis = 0)

cv2.imshow('dst',dst)
cv2.imshow('src',src)
cv2.imshow('number1', number1)
cv2.imshow('number2', number2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## 이미지 연산 2 (그 외)

src = cv2.imread('image/pencil.jpg', 1)
src = cv2.resize(src, dsize = (0,0), fx= 0.2, fy = 0.2, interpolation = cv2.INTER_AREA)
number = np.ones_like(src) * 127

_max = cv2.max(src, number)
_min = cv2.min(src, number)
_abs = cv2.absdiff(src, number)
compare = cv2.compare(src, number, cv2.CMP_GT)

src = np.concatenate((src, src, src, src), axis = 1)
number = np.concatenate((number, number, number, number), axis = 1)
dst = np.concatenate((_max, _min, _abs, compare), axis = 1)

dst = np.concatenate((src, number, dst), axis = 0)

cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## 픽셀 접근

gray = np.linspace(0,255, num = 90000, endpoint = True, retstep = False, dtype = np.uint8).reshape(300,300,1)
color = np.zeros((300, 300, 3), np.uint8)
color[0:150, :, 0] = gray[0:150, :, 0]
color[:, 150:300, 2] = gray[:,150:300,0]
x, y, c = 200, 100, 0
access_gray = gray[y, x, c]
access_color_blue = color[y, x, c]
access_color = color[y, x]

print(access_gray)
print(access_color_blue)
print(access_color)

cv2.imshow("gray", gray)
cv2.imshow("color", color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
b = np.linspace(0, 10, num=5, endpoint=True, retstep=False)
print(b)

In [ ]:
## 트랙바 (이진화, 어파인변환)

def onChange(pos):
    pass

src_gray = cv2.imread('image/cb.jpg', 0) ## 그레이스케일
src_color = cv2.imread('image/cb.jpg', 1) ## bgr 컬러 스케일

src_gray = cv2.resize(src_gray,dsize = (0,0), fx = 0.5, fy = 0.5, interpolation = cv2.INTER_AREA)
src_color = cv2.resize(src_color,dsize = (0,0), fx = 0.5, fy = 0.5, interpolation = cv2.INTER_AREA)

height, width, channel = src_color.shape

cv2.namedWindow("Track_threshold") ## 스레스홀드를 사용하기위한 윈도우 창
cv2.namedWindow("Track_warpAffine") ## 어파인변환을 사용하기 위한 윈도우 창
 
cv2.createTrackbar('threshold', 'Track_threshold', 0, 255, onChange)
cv2.createTrackbar('maxValue', 'Track_threshold', 0, 255 , onChange)

cv2.createTrackbar('rotation', 'Track_warpAffine', 0, 360 , onChange)

cv2.setTrackbarPos('threshold','Track_threshold',127)
cv2.setTrackbarPos('maxValue','Track_threshold',255)
cv2.setTrackbarPos('rotation','Track_warpAffine',180)

while cv2.waitKey(1) != ord('q') :
    
    thresh = cv2.getTrackbarPos('threshold','Track_threshold')
    maxval = cv2.getTrackbarPos('maxValue','Track_threshold')
    _,binary = cv2.threshold(src_gray, thresh, maxval, cv2.THRESH_BINARY)
    
    rot = cv2.getTrackbarPos('rotation','Track_warpAffine')   
    matrix = cv2.getRotationMatrix2D((width/2,height/2), rot, 1)
    warpAffine = cv2.warpAffine(src_color, matrix, (width,height), borderMode = cv2.BORDER_DEFAULT)
    
    cv2.imshow('Track_threshold',binary)
    cv2.imshow('Track_warpAffine',warpAffine)
    
cv2.destroyAllWindows()
    

In [14]:
## 트랙바 (blur, hsv)

def onChange(pos) :
    pass

src = cv2.imread('image/tomato.jpg',1)
src = cv2.resize(src, dsize = (0,0), fx = 0.5, fy = 0.5, interpolation = cv2.INTER_AREA)

hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)

cv2.namedWindow("Track_blur")
cv2.namedWindow("Track_hsv")


cv2.createTrackbar('ksize_x', 'Track_blur', 1, 20, onChange)
cv2.createTrackbar('ksize_y', 'Track_blur', 1, 20, onChange)

cv2.createTrackbar('h_min', 'Track_hsv', 0, 180, onChange)
cv2.createTrackbar('h_max', 'Track_hsv', 0, 180, onChange)

cv2.setTrackbarPos('ksize_x', 'Track_blur', 1)
cv2.setTrackbarPos('ksize_y', 'Track_blur', 1)
cv2.setTrackbarMin('ksize_x', 'Track_blur', 1)
cv2.setTrackbarMin('ksize_y', 'Track_blur', 1)

cv2.setTrackbarPos('h_min', 'Track_hsv', 8)
cv2.setTrackbarPos('h_max', 'Track_hsv', 20)


while cv2.waitKey(1) != ord('q') :
    x = cv2.getTrackbarPos('ksize_x', 'Track_blur')
    y = cv2.getTrackbarPos('ksize_y', 'Track_blur')
    
    blur = cv2.blur(src, (x,y), anchor = (-1,-1))
    

    h_min = cv2.getTrackbarPos('h_min', 'Track_hsv')
    h_max = cv2.getTrackbarPos('h_max', 'Track_hsv')
    
    h, s, v = cv2.split(hsv)
    
    h = cv2.inRange(h, h_min, h_max)
    color = cv2.bitwise_and(hsv,hsv, mask = h)
    color = cv2.cvtColor(color, cv2.COLOR_HSV2BGR)
    
    cv2.imshow("Track_blur", blur)
    cv2.imshow('Track_hsv', color)

cv2.destroyAllWindows()

In [88]:
## 템플릿 검출

src = cv2.imread("image/hats.jpg", cv2.IMREAD_GRAYSCALE)
templit = cv2.imread("image/glass.jpg", cv2.IMREAD_GRAYSCALE)
dst = cv2.imread("image/hats.jpg")

src = cv2.resize(src, dsize = (0,0), fx = 0.4, fy = 0.4, interpolation = cv2.INTER_AREA)
templit = cv2.resize(templit, dsize = (0,0), fx = 0.4, fy = 0.4, interpolation = cv2.INTER_AREA)
dst = cv2.resize(dst, dsize = (0,0), fx = 0.4, fy = 0.4, interpolation = cv2.INTER_AREA)

result = cv2.matchTemplate(src, templit, cv2.TM_SQDIFF_NORMED)

minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(result)
x, y = minLoc
h, w = templit.shape

dst = cv2.rectangle(dst, (x, y), (x +  w, y + h) , (0, 0, 255), 1)
cv2.imshow("dst", dst)
cv2.imshow('src',src)
cv2.imshow('templit', templit)
cv2.waitKey(0)
cv2.destroyAllWindows()